In [4]:
import ctypes
import os
import sys
sys.path.append("..")

In [5]:
from CANapAPI.struct import version_t

In [7]:
# Install CANape. Not included for copyright reasons.
header_file = r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h"
assert os.path.exists(header_file)
CANapAPI_dll= r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll"
assert os.path.exists(CANapAPI_dll)

# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)

## Set the argument and return types.
# Pass by reference.
dll.Asap3GetVersion.argtypes = (ctypes.POINTER(version_t),)
# Return a success bool.
dll.Asap3GetVersion.restype = ctypes.c_bool

#def get_version():
version = version_t()
result = dll.Asap3GetVersion(ctypes.byref(version))

assert version==version
assert version=="2.3.1"
assert version.osRelease==0
assert version.osVersion==b'Windows95/WindowsNT'

# Open CANape in Modal Mode

Modal mode is the best way I've found to reverse engineer this. It allows you to relequish control from Python and do something with the mouse.t

In [9]:
class tAsap3Hdl(ctypes.Structure):
    def __repr__(self):
        return "tAsap3Hdl<>"

TAsap3Hdl = ctypes.POINTER(tAsap3Hdl) # C:\\Program Files (x86)\\Vector CANape 14\\CANapeAPI\\CANapAPI.h: 623

def __repr__(self):
    return "TAsap3Hdl<>"
def byref(self):
    return ctypes.byref(self)

setattr(TAsap3Hdl, "__repr__", __repr__)
setattr(TAsap3Hdl, "byref", property(byref))

handle=TAsap3Hdl()

In [13]:
from cached_property import cached_property

In [14]:
dir(dll_n)

['Asap3GetVersion',
 '_FuncPtr',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_func_flags_',
 '_func_restype_',
 '_handle',
 '_name']

In [24]:
# Install CANape. Not included for copyright reasons.
header_file = r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h"
assert os.path.exists(header_file) or FileNotFoundError("Install CANape 17")
CANapAPI_dll= r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll"
assert os.path.exists(CANapAPI_dll) or FileNotFoundError("Install CANape 17")

class CANapAPI(object):
    def __init__(self, dll_path=CANapAPI_dll):
        self.dll_path = dll_path
    
    @cached_property
    def dll(self):
        dll = ctypes.windll.LoadLibrary(self.dll_path)
        
        ## Set the argument and return types.
        # Pass by reference.
        dll.Asap3GetVersion.argtypes = (ctypes.POINTER(version_t),)
        # Return a success bool.
        dll.Asap3GetVersion.restype = ctypes.c_bool
        
        dll.Asap3Init5.restype = ctypes.c_bool
        dll.Asap3Init5.argtypes = (
            ctypes.POINTER(TAsap3Hdl),
            ctypes.c_ulong,
            ctypes.c_char_p,
            ctypes.c_ulong,
            ctypes.c_ulong,
            ctypes.c_bool,
            ctypes.c_bool,
            ctypes.c_bool,
            ctypes.c_bool
        )
        return dll
    
    @cached_property
    def version(self):
        return self.Asap3GetVersion()

    @cached_property
    def handle(self):
        return TAsap3Hdl()
    
    def Asap3GetVersion(self):
        version = version_t()
        result = dll.Asap3GetVersion(ctypes.byref(version))
        if result:
            return version
        else:
            return None
        
    def Asap3Init5(
        self,
        responseTimeout:int = 10000,
        workingDir:str = os.path.abspath("canape_tmp"),
        fifoSize:int = 8192,
        sampleSize:int = 256,
        debugMode:bool = True,
        clearDeviceList:bool = True,
        bHexmode:bool = False,
        bModalMode:bool = False,
    ):
        """Asap3Init5 Function: Modality
        
        Parameters
        ----------
        responseTimeout : int (ms)
        workingDir: str
        fifoSize: int
        sampleSize: int
        debugMode : bool
        clearDeviceList : bool
        bHexmode : bool
        bModalMode : bool
        
        Returns
        -------
        bool
            Function call success
        """
        workingDir = os.path.abspath(workingDir)
        os.makedirs(workingDir, exist_ok=True)
        
        # Convert to ctypes.
        c_responseTimeout = ctypes.c_ulong()
        c_workingDir = ctypes.c_char_p(workingDir.encode("UTF-8"))
        c_fifoSize = ctypes.c_ulong(fifoSize)
        c_sampleSize = ctypes.c_ulong(sampleSize)
        c_debugMode = ctypes.c_bool(debugMode)
        c_clearDeviceList = ctypes.c_bool(clearDeviceList)
        c_bHexmode = ctypes.c_bool(bHexmode)
        c_bModalMode = ctypes.c_bool(bModalMode)
        
        result = dll.Asap3Init5(
            self.handle.byref,
            c_responseTimeout,
            c_workingDir,
            c_fifoSize,
            c_sampleSize,
            c_debugMode,
            c_clearDeviceList,
            c_bHexmode,
            c_bModalMode,
        )
        return result        

In [25]:
canapapi = CANapAPI()

In [26]:
canapapi.Asap3Init5()

1

In [28]:
canapapi.dll.Asap3Exit2(canapapi.handle, True)

0

In [29]:
canapapi.dll.Asap3Exit(canapapi.handle)

0